In [2]:
!pip install transformers datasets nlp

     |████████████████████████████████| 2.0MB 8.0MB/s 
     |████████████████████████████████| 194kB 41.0MB/s 
     |████████████████████████████████| 1.7MB 35.8MB/s 
     |████████████████████████████████| 890kB 33.5MB/s 
     |████████████████████████████████| 3.2MB 42.1MB/s 
     |████████████████████████████████| 245kB 42.6MB/s 
     |████████████████████████████████| 112kB 53.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=7029927de34829e2d8d6554af551076f691df655b5d5bd2488991a116212afc9
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


# Fine-tuning a model on a text classification task

In [38]:
task = "test"
model_checkpoint = "youscan/ukr-roberta-base"
batch_size = 10

## Loading the dataset

In [9]:
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

import string
import re
import os

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
data = pd.read_csv('/content/drive/MyDrive/contest/data_text/train.csv', index_col=0)
data = data.rename({'title': 'sentence1', 'text': 'sentence2', 'source': 'label'}, axis=1)
data.index = data.index.rename('idx')
data

,sentence1,sentence2,images,label
idx,,,,
1,Міністри Зеленського лишають мільйони пацієнті...,Понад півтора мільйони пацієнтів із тяжкими ді...,NaN,5
2,В АМКУ звертаються щодо незаконної державної д...,За 15 років державна підтримка виробництва ене...,fd92a3dd-1109-49d8-8f5f-eeed72da22ef.png,6
5,У Польщі виявили майже 16 тисяч нових хворих н...,У Польщі виявили 15 698 нових хворих на корона...,f4a284d0-9bb7-4910-8a62-a7bfd0ec29b6.png,2
9,Закриття міжбанку: гривня трохи ослабла,2 жовтня котирування гривні до долара на закри...,NaN,3
10,"У Москві помер український шахіст із бази ""Мир...",У п'ятницю низка російських ЗМІ повідомили про...,"2cbb2339-6dd2-4000-a53c-225ec7aad892.png,d5233...",0
...,...,...,...,...
32366,"""Укрзалізниця"" просить Раду у бюджеті-2021 зак...","""Укрзалізниця"" закликає парламент закласти у д...",NaN,3
32367,Сніг та 20-градусні морози протримаються до ви...,"У вівторок, 16 лютого, в Україні буде без істо...",NaN,2
32368,"Україна хоче оподаткувати Google, Facebook та ...","Гетманцев розповів, як Україна оподатковуватим...",NaN,6


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16177 entries, 1 to 32371
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentence1  16177 non-null  object
 1   sentence2  16177 non-null  object
 2   images     7223 non-null   object
 3   label      16177 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 631.9+ KB


In [13]:
test_data = pd.read_csv('/content/drive/MyDrive/contest/data_text/test.csv', index_col=0)
test_data = test_data.drop(columns=['images'], axis=1).rename({'title': 'sentence1', 'text': 'sentence2', 'source': 'label'}, axis=1)
test_data.index = test_data.index.rename('idx')
test_data

,sentence1,sentence2
idx,,
0,Падіння економіки в Україні не є критичним - ЄБРР,Падіння економічного рівня в Україні не є крит...
3,Геймерам віддають безкоштовно шутер Rage 2 та ...,Шутер Rage 2 віддають безкоштовно в EGS / фото...
4,Записки из Луганска: привычка быть виноватыми ...,"""Ты печешь?"" – спрашивает у меня подруга, пере..."
6,Індекс Dow Jones вперше перевищив 30 тисяч пун...,На тлі новин щодо передачі влади у США та вакц...
7,Трамп і Меркель обговорили підтримку економічн...,Президент США Дональд Трамп і канцлерка Німечч...
...,...,...
32362,США хочуть закупити ще 100 млн доз вакцини про...,Президент США Джо Байден доручив Міністерству ...
32363,Вчителька вдарила школярку по голові і відпові...,У Первомайському ліцеї №7 Харківської області ...
32364,"""Не розслабляємося"": українців просять святкув...",Президент Володимир Зеленський попросив україн...


In [14]:
X, y = data.drop(columns=['label', 'images']), data['label']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [19]:
import datasets
from datasets import Dataset, DatasetDict, load_metric

dataset = datasets.dataset_dict.DatasetDict({'train': Dataset.from_pandas(pd.concat([X, y], axis=1), split='train'), 'validation': Dataset.from_pandas(pd.concat([X_test, y_test], axis=1), split='validation'), 'test': Dataset.from_pandas(test_data, split='test')})
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 16177
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 5339
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'idx'],
        num_rows: 16183
    })
})

## Preprocessing the data

In [20]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
tokenizer("Понад півтора мільйони пацієнтів із тяжкими", "Сніг та 20-градусні морози протримаються.")

{'input_ids': [0, 33280, 14523, 11748, 5620, 677, 5303, 2722, 2, 2, 40793, 308, 342, 1238, 17, 1468, 495, 2942, 16146, 397, 9374, 654, 18, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [23]:
task_to_keys = {
    "test": ("sentence1", "sentence2"),
}

In [24]:
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Text: {dataset['train'][sentence1_key][0]}")
else:
    print(f"Text: {dataset['train'][sentence1_key][0]}")
    print(f"Title: {dataset['train'][sentence2_key][0]}")

Text: Міністри Зеленського лишають мільйони пацієнтів без ліків
Title: Понад півтора мільйони пацієнтів із тяжкими діагнозами ризикують залишитись без ліків у 2020 році, бо МОЗ блокує закупівлі.
Заблоковані рекордні майже 10 мільярдів гривень у бюджеті на закупівлю ліків та медвиробів.
Ключові документи не підписувалися 4 місяці.
З голими руками і неадекватними заявами міністра Ємця: як Україна зустрічає коронавірус
І це в час, коли у світі пандемія коронавірусу, дефіцит багатьох ліків, країни стають в чергу на постачання і рахунок іде не на дні, а на години.
Натомість Україна в цю чергу ще навіть не стала.
З листопада 2019 року був готовий документ – перелік ліків і медвиробів для закупівель – і він весь цей час валявся для підпису на столі то в одного, то в іншого міністра.
Зараз він досі не ухвалений урядом.
Більше того, в міністерстві досі не призначений заступник міністра, який опікуватиметься закупівлями, не працює спеціальна робоча група, без якої закупівлі стоять, – в МОЗ не зр

In [25]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

In [ ]:
preprocess_function(dataset['train'][:5])

{'input_ids': [[0, 42, 8672, 11833, 19881, 1505, 93, 30, 407, 12321, 281, 11, 17841, 11842, 10668, 6108, 16, 2106, 14483, 2292, 9008, 39246, 2, 2, 39675, 535, 15280, 6108, 16, 2106, 14483, 7390, 840, 310, 2922, 1118, 3502, 225, 203, 5506, 17, 11620, 2428, 336, 2189, 1259, 315, 527, 44360, 2735, 8286, 3137, 7923, 320, 5077, 1362, 16, 399, 10265, 14194, 775, 840, 26012, 4388, 347, 18, 3059, 34560, 7370, 358, 14734, 16, 2110, 42079, 327, 8784, 1329, 1029, 5734, 874, 18, 203, 29393, 2569, 5048, 362, 8577, 10039, 462, 4576, 1756, 7305, 565, 2605, 3020, 412, 21629, 295, 2189, 1259, 315, 527, 22849, 32748, 1674, 284, 19599, 8175, 18, 3059, 3253, 5927, 284, 14700, 2673, 1390, 16, 34390, 1024, 22477, 6378, 16, 603, 4137, 15828, 2163, 379, 448, 42703, 16, 471, 5214, 1905, 8672, 11833, 19881, 1505, 93, 18, 32074, 556, 674, 26788, 1166, 6, 2264, 2843, 17, 288, 1631, 3733, 281, 3187, 490, 3582, 602, 281, 11, 17841, 556, 21430, 324, 1167, 2110, 926, 281, 6334, 4665, 14300, 469, 7471, 6254, 39314, 19

In [26]:
encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2'],
        num_rows: 16177
    })
    validation: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2'],
        num_rows: 5339
    })
    test: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'sentence1', 'sentence2'],
        num_rows: 16183
    })
})

In [ ]:
encoded_dataset["validation"][0]

{'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  

## Fine-tuning the model

In [27]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, TFBertForSequenceClassification
num_labels = len(data['label'].unique())
metric_name = "f1"
validation_key = "validation"

In [28]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
model

Some weights of the model checkpoint at youscan/ukr-roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at youscan/ukr-roberta-base and are newly ini

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [39]:
args = TrainingArguments(
    "test-BERT-ukr",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=4,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

In [40]:
from sklearn.metrics import f1_score

def compute_metrics(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=1)

    f1 = f1_score(labels, predictions, average='macro')
    return {
        'f1': f1,
    }

In [41]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [42]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1,Runtime,Samples Per Second
1,0.183000,0.111959,0.963589,199.678000,26.738000
2,0.104200,0.059625,0.977487,199.083400,26.818000
3,0.055600,0.030175,0.988063,198.892800,26.844000
4,0.030700,0.020766,0.992061,198.994500,26.830000


TrainOutput(global_step=6472, training_loss=0.10995115617886463, metrics={'train_runtime': 7956.4231, 'train_samples_per_second': 0.813, 'total_flos': 2.495379979803612e+16, 'epoch': 4.0, 'init_mem_cpu_alloc_delta': 281900, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 729635, 'init_mem_gpu_peaked_delta': 1016655872, 'train_mem_cpu_alloc_delta': 1999769, 'train_mem_gpu_alloc_delta': 1526233088, 'train_mem_cpu_peaked_delta': 160262829, 'train_mem_gpu_peaked_delta': 7679032320})

In [ ]:
trainer.evaluate()

In [43]:
res = trainer.predict(encoded_dataset['test'])
res

PredictionOutput(predictions=array([[-3.0266078 , -2.2053356 , -1.8662642 , ..., -0.92149943,
        -2.17972   , -1.8585972 ],
       [-1.4444575 , -1.7879773 , -1.4425111 , ..., -1.7304399 ,
        -1.3040822 , 10.336172  ],
       [-0.94373024, -1.6593742 , -1.4136195 , ..., -0.72311115,
         9.707149  , -1.8310527 ],
       ...,
       [ 9.415085  , -1.0328062 , -1.3470451 , ..., -1.4317446 ,
        -0.76637703, -2.3409975 ],
       [-2.602948  , -1.9182569 ,  0.16874278, ...,  8.677182  ,
        -1.5783421 , -2.0681076 ],
       [ 9.445286  , -2.0399036 , -1.5063915 , ..., -0.14742588,
        -1.7164369 , -1.0523508 ]], dtype=float32), label_ids=None, metrics={'eval_runtime': 603.893, 'eval_samples_per_second': 26.798, 'test_mem_cpu_alloc_delta': 533650, 'test_mem_gpu_alloc_delta': 0, 'test_mem_cpu_peaked_delta': 571266, 'test_mem_gpu_peaked_delta': 369695744})

In [44]:
df_preds = pd.DataFrame(res.predictions, index=test_data.index)
df_preds

,0,1,2,3,4,5,6
idx,,,,,,,
0,-3.026608,-2.205336,-1.866264,9.459106,-0.921499,-2.179720,-1.858597
3,-1.444458,-1.787977,-1.442511,-1.638201,-1.730440,-1.304082,10.336172
4,-0.943730,-1.659374,-1.413620,-1.096715,-0.723111,9.707149,-1.831053
6,-2.776175,-1.261119,-2.278135,9.443981,-1.386909,-1.835468,-2.215858
7,-2.377481,-2.297837,2.360936,-2.072962,8.464268,-2.379967,-2.324723
...,...,...,...,...,...,...,...
32362,-1.780088,10.189858,-1.608617,-2.056852,-1.668188,-1.486978,-2.043016
32363,-0.766646,-2.172107,-0.881372,-1.192516,-2.283178,9.468607,0.052802
32364,9.415085,-1.032806,-1.347045,-2.381353,-1.431745,-0.766377,-2.340997


In [45]:
df_preds.to_csv('./bert-predictions-4.csv')

In [ ]:
df_res = pd.DataFrame(np.argmax(res.predictions, axis=1), index=test_data.index)
df_res.index = df_res.index.rename('Id')
df_res.columns = ['Predicted']
df_res

,Predicted
Id,
0,3
3,6
4,5
6,3
7,4
...,...
32362,1
32363,5
32364,0


In [ ]:
df_res.to_csv('./submission-ukr-bert-base-3.csv')

In [ ]:
! cat /content/test-BERT-ukr/checkpoint-6069/trainer_state.json

{
  "best_metric": 0.9872465169166895,
  "best_model_checkpoint": "test-BERT-ukr/checkpoint-6069",
  "epoch": 3.0,
  "global_step": 6069,
  "is_hyper_param_search": false,
  "is_local_process_zero": true,
  "is_world_process_zero": true,
  "log_history": [
    {
      "epoch": 0.25,
      "learning_rate": 1.901136925358379e-05,
      "loss": 0.5381,
      "step": 500
    },
    {
      "epoch": 0.49,
      "learning_rate": 1.8022738507167574e-05,
      "loss": 0.2358,
      "step": 1000
    },
    {
      "epoch": 0.74,
      "learning_rate": 1.7034107760751362e-05,
      "loss": 0.1882,
      "step": 1500
    },
    {
      "epoch": 0.99,
      "learning_rate": 1.6045477014335146e-05,
      "loss": 0.2048,
      "step": 2000
    },
    {
      "epoch": 1.0,
      "eval_f1": 0.9633668707214312,
      "eval_loss": 0.11797618865966797,
      "eval_runtime": 202.3295,
      "eval_samples_per_second": 26.388,
      "step": 2023
    },
    {
      "epoch": 1.24,
      "learning_rate": 1.505

In [ ]:
! mv /content/test-BERT-ukr/checkpoint-6069 /content/drive/MyDrive/contest/models/ukr-BERT/checkpoint-6069

In [ ]:
! mv 

In [ ]:
model_checkpoint = '../input/xlmroberta3/test-RoBERTa/checkpoint-16179'
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

In [ ]:
args = TrainingArguments(
    "test-RoBERTa-v2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

new_trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
model

In [ ]:
new_trainer.train()

In [ ]:
res = new_trainer.predict(encoded_dataset['test'])

df_res = pd.DataFrame(np.argmax(res.predictions, axis=1), index=test_data.index)
df_res.index = df_res.index.rename('Id')
df_res.columns = ['Predicted']

df_res.to_csv('./submission-xlm-roberta-large-4.csv')

In [ ]:
features = new_trainer.predict(Dataset.from_dict(encoded_dataset['test'][:100])) # Dataset.from_dict({'input_ids': input_ids, 'attention_mask': attention_mask}) .remove_columns('label')
features

In [ ]:
features.predictions.shape

In [ ]:
def extract_features(data, size):
    res_features = []
    offset = 0
    count = 1000
    while True:
        features = new_trainer.predict(Dataset.from_dict(data[offset:offset + count]))
        res_features.append(np.apply_over_axes(np.mean, features.predictions, [1,]))
        offset += count
        if len(data) < offset:
            break
        print(f'{offset}/{size}')
    return res_features

In [ ]:
test_features = np.array([])
for feat in res_features:
    test_features = np.append(test_features, feat)
    print(test_features.shape)
test_features = test_features.reshape((len(test_data), 1024))
test_features.shape

In [ ]:
df_res = pd.DataFrame(test_features, index=test_data.index)
df_res

In [ ]:
df_res.info()

In [ ]:
df_res.to_csv('./test-features.csv')

In [ ]:
res_features = extract_features(encoded_dataset['train'].remove_columns('label'), len(data))
res_features

In [ ]:
train_features = np.array([])
for feat in res_features:
    train_features = np.append(train_features, feat)
    print(train_features.shape)
train_features = train_features.reshape((len(data), 1024))
train_features.shape

In [ ]:
df_train = pd.DataFrame(train_features, index=data.index)
df_train

In [ ]:
df_train.to_csv('./train-features.csv')

In [ ]:
import shutil

shutil.copytree('./test-RoBERTa', '/content/drive/MyDrive/contest/models/RoBERTa-large')

In [ ]:
! rm -rf ./test-RoBERTa/checkpoint-5393/

In [ ]:
!zip -r ./RoBERTa.zip ./test-RoBERTa/checkpoint-16179/

<a href="./RoBERTa.zip"> Download File </a>

In [ ]:
model_checkpoint = ''

In [ ]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

And we can instantiate our `Trainer` like before:

In [ ]:
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

The method we call this time is `hyperparameter_search`. Note that it can take a long time to run on the full dataset for some of the tasks. You can try to find some good hyperparameter on a portion of the training dataset by replacing the `train_dataset` line above by:
```python
train_dataset = encoded_dataset["train"].shard(index=1, num_shards=10) 
```
for 1/10th of the dataset. Then you can run a full training on the best hyperparameters picked by the search.

In [ ]:
best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize")

The `hyperparameter_search` method returns a `BestRun` objects, which contains the value of the objective maximized (by default the sum of all metrics) and the hyperparameters it used for that run.

In [ ]:
best_run

You can customize the objective to maximize by passing along a `compute_objective` function to the `hyperparameter_search` method, and you can customize the search space by passing a `hp_space` argument to `hyperparameter_search`. See this [forum post](https://discuss.huggingface.co/t/using-hyperparameter-search-in-trainer/785/10) for some examples.

To reproduce the best training, just set the hyperparameters in your `TrainingArgument` before creating a `Trainer`:

In [ ]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()

Don't forget to [update your model](https://huggingface.co/transformers/model_sharing.html) on the [🤗 Model Hub](https://huggingface.co/models). You can then use it only to generate results like the one shown in the first picture of this notebook!

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session